In [15]:
import json
from pathlib import Path

# Path to train.json (convert string to Path)
train_path = Path(
    "C:/Users/JOHN EZE/Desktop/Research/chichewa-text2sql/data/train/train.json"
)

# Load train.json

with train_path.open("r", encoding="utf-8") as f:
    train_data = json.load(f)

# Sanity check
print(f"Loaded {len(train_data)} training examples")
train_data[0]


Loaded 400 training examples


{'id': 1,
 'question_en': 'Which district produced the most Maize',
 'question_ny': 'Ndi boma liti komwe anakolola chimanga chambiri?',
 'sql_statement': "WITH max_yield_district AS (\n    SELECT district\n    FROM production\n    WHERE crop = 'Maize'\n    ORDER BY yield DESC\n    LIMIT 1\n)\nSELECT district\nFROM max_yield_district;",
 'sql_result': "[('Lilongwe',)]",
 'difficulty_level': 'hard',
 'table': 'production'}

In [16]:
import pandas as pd

df = pd.DataFrame(train_data)

df.head()


,id,question_en,question_ny,sql_statement,sql_result,difficulty_level,table
0,1,Which district produced the most Maize,Ndi boma liti komwe anakolola chimanga chambiri?,WITH max_yield_district AS (\n SELECT distr...,"[('Lilongwe',)]",hard,production
1,2,Where was the most rice harvested?,Ndi boma liti komwe anakolola rishi chambiri?,WITH max_yield_district AS (\n SELECT distr...,"[('Karonga',)]",hard,production
2,3,Which are the top 5 maize producing districts?,Ndi maboma ati asanu omwe amakolola chimanga c...,WITH max_yield_district AS (\n SELECT distr...,"[('Lilongwe',), ('Kasungu',), ('Dowa',), ('Mzi...",hard,production
3,4,Which crops did well in Rumphi in terms of yie...,Ndi mbeu zanji zisanu zapamwamba anakolola bwi...,SELECT crop FROM production WHERE district = '...,"[('Cassava',), ('Maize',), ('Sweet potatoes',)...",medium,production
4,5,Whichare the top 5 districts that harvested th...,Ndi maboma ati asanu omwe anakolola kachipiri ...,WITH max_yield_district AS (\n SELECT distr...,"[('Lilongwe',), ('Dowa',), ('Nkhotakota',), ('...",hard,production


In [17]:
df.sample(10)

,id,question_en,question_ny,sql_statement,sql_result,difficulty_level,table
11,12,How much Maize was harvested Ntcheu?,Ndi chimanga chochuluka bwanj chidakololedwa m...,SELECT Yield FROM production WHERE District = ...,"[(81248.0,)]",easy,production
236,237,Food insecurity percentage in Rumphi?,Peresenti ya kusowa chakudya ku Rumphi?,SELECT percentage_population FROM food_insecur...,"[(65,), (25,), (10,), (0,), (0,), (75,), (20,)...",easy,food_insecurity
243,244,Where is it cheap to buy Maize?,Ndingakagule kuti chimanga chotchipa?,WITH latest_date AS (\n SELECT MAX(collecti...,"[('Bangula', None), ('Bowe', None), ('Bvumbwe'...",hard,commodity_prices
151,152,What is the total population in CHIRADZULU?,Ndi anthu angati ku CHIRADZULU?,SELECT SUM(CAST(total_population AS INTEGER)) ...,"[(53477,)]",easy,population
301,302,Which commodity shows the most stable prices i...,Ndi katundu uti yemwe akuwonetsa kusakhazikika...,SELECT commodity FROM commodity_prices WHERE d...,"[('Rice',)]",hard,commodity_prices
46,47,How much Maize in total was harvested in the c...,Ndi chimanga chochuluka bwanji chidakololedwa ...,SELECT SUM(Yield) as Total_Yield FROM producti...,"[(3111592.95,)]",easy,production
371,372,Which company had the lowest average trading v...,Ndi kampani iti yomwe inali ndi kuchuluka koch...,SELECT company_name FROM mse_daily GROUP BY co...,"[('Sunbird Tourism',)]",medium,mse_daily
146,147,List TAs in Neno,Mafumu ku Neno,SELECT DISTINCT ta_name FROM population WHERE ...,"[('TA Chekucheku',), ('Neno Boma',), ('TA Symo...",easy,population
276,277,Which commodity had prices recorded in all dis...,Ndi chakudya chiti chomwe chinalembedwa miteng...,SELECT commodity FROM commodity_prices GROUP B...,"[('Beans',), ('Cow peas',), ('Groundnuts',), (...",hard,commodity_prices
155,156,How many females in Blantyre District?,Ndi akazi angati ku Blantyre District?,SELECT SUM(population_female) FROM population ...,"[(36676.0,)]",easy,population


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                400 non-null    int64 
 1   question_en       400 non-null    object
 2   question_ny       400 non-null    object
 3   sql_statement     400 non-null    object
 4   sql_result        400 non-null    object
 5   difficulty_level  400 non-null    object
 6   table             400 non-null    object
dtypes: int64(1), object(6)
memory usage: 22.0+ KB


In [20]:
df.table.value_counts()

table
production          80
population          80
food_insecurity     80
commodity_prices    80
mse_daily           80
Name: count, dtype: int64

In [6]:
valid_tables = {
    "production",
    "commodity_prices",
    "population",
    "mse_daily",
    "food_insecurity"
}

set(df["table"]) - valid_tables


set()

In [7]:
# Function to test a single train example
def test_sql_instance(instance):
    sql_query = instance["sql_statement"]
    try:
        cursor.execute(sql_query)
        result = cursor.fetchall()
        print(f"Query: {sql_query}\nResult: {result}\n")
    except Exception as e:
        print(f"Query: {sql_query}\nError: {e}\n")

# Test each instance one by one
for i, instance in enumerate(train_data):
    print(f"Example {i+1}:")
    test_sql_instance(instance)
    # Optionally, break after first few for sanity check
    # if i > 4: break

Example 1:
Query: WITH max_yield_district AS (
    SELECT district
    FROM production
    WHERE crop = 'Maize'
    ORDER BY yield DESC
    LIMIT 1
)
SELECT district
FROM max_yield_district;
Error: name 'cursor' is not defined

Example 2:
Query: WITH max_yield_district AS (
    SELECT district
    FROM production
    WHERE crop = 'Rice'
    ORDER BY yield DESC
    LIMIT 1)
SELECT district
FROM max_yield_district;
Error: name 'cursor' is not defined

Example 3:
Query: WITH max_yield_district AS (
    SELECT district
    FROM production
    WHERE crop = 'Maize'
    ORDER BY yield DESC
    LIMIT 5)
SELECT district
FROM max_yield_district;
Error: name 'cursor' is not defined

Example 4:
Query: SELECT crop FROM production WHERE district = 'Rumphi' ORDER BY yield DESC LIMIT 5;
Error: name 'cursor' is not defined

Example 5:
Query: WITH max_yield_district AS (
    SELECT district
    FROM production
    WHERE crop = 'Tobacco'
    ORDER BY yield DESC
    LIMIT 5)
SELECT district
FROM max_yield

In [8]:
# ============================================================================
# COMPREHENSIVE VERIFICATION OF CORRECTED TRAIN.JSON
# ============================================================================
# This script verifies that every SQL statement in train_corrected.json
# executes correctly and produces the expected sql_result

import json
import sqlite3
import os
from typing import Tuple, Any

# Paths
BASE_DIR = os.path.dirname(os.getcwd())
DATABASE_PATH = os.path.join(BASE_DIR, "data", "database", "chichewa_text2sql.db")
DATASET_PATH = os.path.join(BASE_DIR, "data", "train", "train.json")

print(f"Database: {DATABASE_PATH}")
print(f"Dataset: {DATASET_PATH}")
print(f"Database exists: {os.path.exists(DATABASE_PATH)}")
print(f"Dataset exists: {os.path.exists(DATASET_PATH)}")

Database: c:\Users\JOHN EZE\Desktop\Research\chichewa-text2sql\data\database\chichewa_text2sql.db
Dataset: c:\Users\JOHN EZE\Desktop\Research\chichewa-text2sql\data\train\train.json
Database exists: True
Dataset exists: True


In [9]:
# Load the corrected dataset
with open(DATASET_PATH, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

print(f"Total instances loaded: {len(dataset)}")
print(f"\nSample instance:")
print(json.dumps(dataset[0], indent=2))

Total instances loaded: 400

Sample instance:
{
  "question_en": "Which district produced the most Maize",
  "question_ny": "Ndi boma liti komwe anakolola chimanga chambiri?",
  "sql_statement": "WITH max_yield_district AS (\n    SELECT district\n    FROM production\n    WHERE crop = 'Maize'\n    ORDER BY yield DESC\n    LIMIT 1\n)\nSELECT district\nFROM max_yield_district;",
  "sql_result": "[('Lilongwe',)]",
  "difficulty_level": "hard",
  "table": "production"
}


In [10]:
# Connect to the database
conn = sqlite3.connect(DATABASE_PATH)
cursor = conn.cursor()

def execute_sql(sql: str) -> Tuple[bool, Any, str]:
    """Execute SQL and return (success, result, error_message)."""
    try:
        cursor.execute(sql)
        result = cursor.fetchall()
        return True, result, ""
    except sqlite3.Error as e:
        return False, None, str(e)

def format_result_for_comparison(result) -> str:
    """Format result the same way as in the dataset for comparison."""
    if result is None:
        return "[]"
    if not result:
        return "[]"
    
    formatted = []
    for row in result:
        formatted_row = []
        for val in row:
            if isinstance(val, float):
                formatted_row.append(round(val, 2))
            else:
                formatted_row.append(val)
        formatted.append(tuple(formatted_row))
    
    return str(formatted)

print("Database connection established.")

Database connection established.


In [11]:
# ============================================================================
# VERIFY EACH INSTANCE ONE BY ONE
# ============================================================================

print("=" * 80)
print("VERIFICATION RESULTS - CHECKING EVERY INSTANCE")
print("=" * 80)

valid_count = 0
mismatch_count = 0
error_count = 0

mismatches = []
errors = []

for i, instance in enumerate(dataset):
    sql = instance['sql_statement']
    expected = instance['sql_result']
    table = instance['table']
    question = instance['question_en']
    
    # Execute the SQL
    success, actual_result, error_msg = execute_sql(sql)
    
    if not success:
        error_count += 1
        errors.append({
            'index': i,
            'table': table,
            'question': question,
            'sql': sql,
            'error': error_msg
        })
        print(f"[{i:3d}] ❌ ERROR - {table}: {error_msg[:50]}...")
        continue
    
    # Format and compare
    actual_formatted = format_result_for_comparison(actual_result)
    
    if actual_formatted == expected:
        valid_count += 1
        print(f"[{i:3d}] ✓ VALID - {table}: {question[:50]}...")
    else:
        mismatch_count += 1
        mismatches.append({
            'index': i,
            'table': table,
            'question': question,
            'sql': sql,
            'expected': expected,
            'actual': actual_formatted
        })
        print(f"[{i:3d}] ⚠ MISMATCH - {table}: {question[:50]}...")

print("\n" + "=" * 80)
print("VERIFICATION SUMMARY")
print("=" * 80)
print(f"Total instances:    {len(dataset)}")
print(f"Valid:              {valid_count} ✓")
print(f"Mismatches:         {mismatch_count} ⚠")
print(f"Execution errors:   {error_count} ❌")
print(f"Success rate:       {valid_count / len(dataset) * 100:.2f}%")
print("=" * 80)

VERIFICATION RESULTS - CHECKING EVERY INSTANCE
[  0] ✓ VALID - production: Which district produced the most Maize...
[  1] ✓ VALID - production: Where can I find alot of rice to purchase?...
[  2] ✓ VALID - production: Where can I find Maize?...
[  3] ✓ VALID - production: Which crops did well in Rumphi?...
[  4] ✓ VALID - production: Which districts harvested the most tobacco?...
[  5] ✓ VALID - production: Where can I buy soya?...
[  6] ✓ VALID - production: Which district produced more maize: Lilongwe or Ka...
[  7] ✓ VALID - production: Which district produced the most Macademia in the ...
[  8] ✓ VALID - production: How much Apples were harvested in Dowa during the ...
[  9] ✓ VALID - production: What was the total yield of Velvet beans in the 20...
[ 10] ✓ VALID - production: How much Wheat was produced in Nkhotakota?...
[ 11] ✓ VALID - production: How much Maize was harvested Ntcheu?...
[ 12] ✓ VALID - production: Which crop had the highest yield in Ntchisi during...
[ 13] ✓ VAL

In [ ]:
# ============================================================================
# SHOW DETAILS OF ANY MISMATCHES
# ============================================================================

if mismatches:
    print("\n" + "=" * 80)
    print("MISMATCH DETAILS")
    print("=" * 80)
    
    for m in mismatches[:10]:  # Show first 10 mismatches
        print(f"\n[Instance {m['index']}] Table: {m['table']}")
        print(f"Question: {m['question']}")
        print(f"SQL: {m['sql'][:100]}...")
        print(f"Expected: {m['expected'][:100]}...")
        print(f"Actual:   {m['actual'][:100]}...")
        print("-" * 80)
else:
    print("\n✅ ALL INSTANCES VERIFIED SUCCESSFULLY - NO MISMATCHES!")

if errors:
    print("\n" + "=" * 80)
    print("EXECUTION ERROR DETAILS")
    print("=" * 80)
    
    for e in errors[:10]:  # Show first 10 errors
        print(f"\n[Instance {e['index']}] Table: {e['table']}")
        print(f"Question: {e['question']}")
        print(f"SQL: {e['sql'][:100]}...")
        print(f"Error: {e['error']}")
        print("-" * 80)
else:
    print("\n✅ ALL SQL STATEMENTS EXECUTED SUCCESSFULLY - NO ERRORS!")


MISMATCH DETAILS

[Instance 0] Table: production
Question: Which district produced the most Maize
SQL: WITH max_yield_district AS (
    SELECT district
    FROM production
    WHERE crop = 'Maize'
    OR...
Expected: Lilongwe...
Actual:   [('Lilongwe',)]...
--------------------------------------------------------------------------------

[Instance 1] Table: production
Question: Where can I find alot of rice to purchase?
SQL: WITH max_yield_district AS (
    SELECT district
    FROM production
    WHERE crop = 'Rice'
    ORD...
Expected: Karonga...
Actual:   [('Karonga',)]...
--------------------------------------------------------------------------------

[Instance 2] Table: production
Question: Where can I find Maize?
SQL: WITH max_yield_district AS (
    SELECT district
    FROM production
    WHERE crop = 'Maize'
    OR...
Expected: [["Lilongwe"], ["Kasungu"], ["Dowa"], ["Mzimba"], ["Mchinji"]]...
Actual:   [('Lilongwe',), ('Kasungu',), ('Dowa',), ('Mzimba',), ('Mchinji',)]...
----

: 

In [12]:
# ============================================================================
# VERIFICATION BY TABLE
# ============================================================================

print("\n" + "=" * 80)
print("VERIFICATION BY TABLE")
print("=" * 80)

from collections import defaultdict
table_stats = defaultdict(lambda: {'valid': 0, 'mismatch': 0, 'error': 0, 'total': 0})

for i, instance in enumerate(dataset):
    table = instance['table']
    table_stats[table]['total'] += 1

# Count from our results
for e in errors:
    table_stats[e['table']]['error'] += 1
    
for m in mismatches:
    table_stats[m['table']]['mismatch'] += 1

for table in table_stats:
    table_stats[table]['valid'] = (table_stats[table]['total'] - 
                                    table_stats[table]['error'] - 
                                    table_stats[table]['mismatch'])

print(f"\n{'Table':<20} {'Total':>8} {'Valid':>8} {'Mismatch':>10} {'Error':>8} {'Rate':>10}")
print("-" * 70)

for table, stats in sorted(table_stats.items()):
    rate = stats['valid'] / stats['total'] * 100 if stats['total'] > 0 else 0
    status = "✓" if rate == 100 else "⚠"
    print(f"{table:<20} {stats['total']:>8} {stats['valid']:>8} {stats['mismatch']:>10} {stats['error']:>8} {rate:>9.1f}% {status}")

print("-" * 70)


VERIFICATION BY TABLE

Table                   Total    Valid   Mismatch    Error       Rate
----------------------------------------------------------------------
commodity_prices           80       80          0        0     100.0% ✓
food_insecurity            80       80          0        0     100.0% ✓
mse_daily                  80       80          0        0     100.0% ✓
population                 80       80          0        0     100.0% ✓
production                 80       80          0        0     100.0% ✓
----------------------------------------------------------------------


In [13]:
# ============================================================================
# SPOT CHECK - RANDOMLY VERIFY 5 INSTANCES MANUALLY
# ============================================================================

import random
random.seed(42)

print("\n" + "=" * 80)
print("SPOT CHECK - DETAILED MANUAL VERIFICATION OF 5 RANDOM INSTANCES")
print("=" * 80)

sample_indices = random.sample(range(len(dataset)), 5)

for idx in sample_indices:
    instance = dataset[idx]
    print(f"\n{'='*80}")
    print(f"INSTANCE #{idx}")
    print(f"{'='*80}")
    print(f"Table: {instance['table']}")
    print(f"Difficulty: {instance['difficulty_level']}")
    print(f"\nQuestion (EN): {instance['question_en']}")
    print(f"Question (NY): {instance['question_ny']}")
    print(f"\nSQL Statement:")
    print(f"  {instance['sql_statement']}")
    
    # Execute
    success, result, error = execute_sql(instance['sql_statement'])
    actual_formatted = format_result_for_comparison(result) if success else "ERROR"
    
    print(f"\nStored Result:   {instance['sql_result'][:100]}...")
    print(f"Executed Result: {actual_formatted[:100]}...")
    
    match = "✓ MATCH" if actual_formatted == instance['sql_result'] else "✗ MISMATCH"
    print(f"\nVerification: {match}")


SPOT CHECK - DETAILED MANUAL VERIFICATION OF 5 RANDOM INSTANCES

INSTANCE #327
Table: mse_daily
Difficulty: easy

Question (EN): What is the average closing price of Sunbird Tourism?
Question (NY): Mtengo wapakati wotseka wa Sunbird Tourism ndi uti?

SQL Statement:
  SELECT AVG(close_price) FROM mse_daily WHERE company_name = 'Sunbird Tourism';

Stored Result:   [(173.27,)]...
Executed Result: [(173.27,)]...

Verification: ✓ MATCH

INSTANCE #57
Table: production
Difficulty: hard

Question (EN): Which district had the highest number of different crops produced in the 2023-2024 season?
Question (NY): Ndi boma liti lomwe linakolola mitundu yambiri ya mbewu mu nyengo ya 2023-2024?

SQL Statement:
  SELECT district FROM production WHERE season = '2023-2024' GROUP BY district ORDER BY COUNT(DISTINCT crop) DESC LIMIT 1;

Stored Result:   [('mwanza',)]...
Executed Result: [('mwanza',)]...

Verification: ✓ MATCH

INSTANCE #12
Table: production
Difficulty: easy

Question (EN): Which crop had th

In [14]:
# ============================================================================
# FINAL REPORT
# ============================================================================

print("\n" + "=" * 80)
print("FINAL VERIFICATION REPORT")
print("=" * 80)

print(f"""
Dataset: train_corrected.json
Database: chichewa_text2sql.db

STATISTICS:
-----------
Total Instances:     {len(dataset)}
Successfully Valid:  {valid_count}
Mismatches:          {mismatch_count}
Execution Errors:    {error_count}

SUCCESS RATE: {valid_count / len(dataset) * 100:.2f}%

TABLES COVERED:
""")

for table in sorted(table_stats.keys()):
    print(f"  - {table}: {table_stats[table]['total']} instances")

if valid_count == len(dataset):
    print("\n" + "🎉" * 20)
    print("CONGRATULATIONS! ALL 400 INSTANCES ARE VERIFIED AND CORRECT!")
    print("The dataset is ready for Text-to-SQL experiments.")
    print("🎉" * 20)
else:
    print(f"\n⚠️  {len(dataset) - valid_count} instances need attention.")

# Close database connection
conn.close()
print("\nDatabase connection closed.")


FINAL VERIFICATION REPORT

Dataset: train_corrected.json
Database: chichewa_text2sql.db

STATISTICS:
-----------
Total Instances:     400
Successfully Valid:  400
Mismatches:          0
Execution Errors:    0

SUCCESS RATE: 100.00%

TABLES COVERED:

  - commodity_prices: 80 instances
  - food_insecurity: 80 instances
  - mse_daily: 80 instances
  - population: 80 instances
  - production: 80 instances

🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉
CONGRATULATIONS! ALL 400 INSTANCES ARE VERIFIED AND CORRECT!
The dataset is ready for Text-to-SQL experiments.
🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

Database connection closed.
